<a href="https://colab.research.google.com/github/peshqa/MMO_lab/blob/main/lab6_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл (текстовый)
* .bin.gz — бинарный файл

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`.

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. **для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она**.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г.
Полный список моделей тут: https://rusvectores.org/ru/models/

Для начала импортируем необходимые библиотеки:

In [1]:
import urllib.request # библиотека для скачивания данных
import gensim # библиотека для загрузки и использвоания моделй w2v
from gensim.models import word2vec # непосредственно методы w2v


In [27]:
# скачиваем модель ruscorpora_mystem_cbow_300 с сайта rusvectores
# 300 - размерность вектора embeddings для слов

urllib.request.urlretrieve("https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz", "ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz")

KeyboardInterrupt: 

In [3]:
model_path = 'ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path)

Посмотрим на ближайших соседей следующей группы слов:

In [4]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'биткоин_NOUN']

Частеречные тэги (например, _NOUN, тег части речи слова) нужны, поскольку это специфика скачанной модели - она была натренирована на словах, размеченных по частям речи (и лемматизированных).

**NB!** В названиях моделей на `rusvectores` указано, какой тегсет (набор обозначений тегов) они используют (mystem, upos и т.д.)
расшифровка частей речи тут
https://yandex.ru/dev/mystem/doc/grammemes-values.html/#grammemes-values__parts

Попросим у модели 10 ближайших соседей для каждого слова и косинусные близости для каждого:

In [5]:
for word in words:
    # есть ли слово в модели?
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_NOUN
[-0.028439 -0.037945  0.011242 -0.021425  0.057268  0.071456 -0.001182
  0.026486 -0.033046 -0.019704]
неделя_NOUN :  0.7097942233085632
месяц_NOUN :  0.6908209919929504
утро_NOUN :  0.6243225932121277
днемя_NOUN :  0.6224159598350525
днями_NOUN :  0.6100884675979614
воскресенье_NOUN :  0.6087555885314941
вечер_NOUN :  0.6030082702636719
дня_NOUN :  0.592153012752533
час_NOUN :  0.590473473072052
сутки_NOUN :  0.5869504809379578


ночь_NOUN
[ 0.046085  0.00211   0.065384  0.010745 -0.035544  0.053312  0.084506
  0.005005 -0.037686 -0.006833]
ночь_PROPN :  0.7704508304595947
вечер_NOUN :  0.7683228254318237
утро_NOUN :  0.7520124316215515
полночь_NOUN :  0.7201331853866577
рассвет_NOUN :  0.6792924404144287
полдень_NOUN :  0.6637035012245178
утро_PROPN :  0.6531521677970886
ночь_ADV :  0.6248846650123596
сумерки_NOUN :  0.6227153539657593
ночью_NOUN :  0.6219336986541748


человек_NOUN
[-0.018347  0.00314   0.100693 -0.020991 -0.028008  0.075846 -0.105468
 -0.043332 -0.042502 

Находить косинусную близость пары слов функцией ```similarity()```:

In [6]:
print(model_ru.similarity('студент_NOUN', 'преподаватель_NOUN'))

0.5173436


У загруженной модели много различных функций. Например, можно решать задачи на семантическую близость.

Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

Для решения примера в качестве параметров метода ```most_similar()``` необходимо передать:
* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

*Замечание:* не забываем взять самый близкий элемент, для этого необходимо указать ```[0][0]```.

In [7]:
print(model_ru.most_similar(positive=['пицца_NOUN', 'сибирь_NOUN'], negative=['италия_NOUN'])[0][0])

пельмень_NOUN


In [8]:
# придумайте и проверьте с помощью метода most_similar какую-нибудь  аналогию



Метод ```doesnt_match()``` находит "лишнее слово" в группе слов:

In [9]:
model_ru.doesnt_match(words)

'семантика_NOUN'

#  Применим полученные выше навыки и решим простую задачу анализа тональности твиттов:

Проделаем весь пайплайн от сырых текстов до получения обученной модели.
Отдельно скачиваем файлы с положительно окрашенными твитами и негативно окрашеннными.
Это реальные данные русскоязычного сегмента твиттера.
https://study.mokoron.com/



In [10]:
!wget -O positive.csv https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

--2024-04-22 19:02:04--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2024-04-22 19:02:05--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc64ad4108b19bb60f4164057dfd.dl.dropboxusercontent.com/cd/0/inline/CRjEiPCmtVpe0_L6wTn4ueBisEU8-lELt5O-xcb84srR7e3FLMgRwgsKMGDMmCZACbJLVh7uT8W3Smp2lBwB2pOPn_GztDxRxl59A65m-B4l0dfI9v8rS01dYipmmDvkfvM/file# [following]
--2024-04-22 19:02:05--  https://uc64ad4108b19bb60f4164057dfd.dl.dropboxusercontent.com/cd/0/inline/CRjEiPCmtVpe0_L6wTn4ueBisEU8-lELt5O-xcb84srR7e3FLMgRwgsKMGDMmCZACbJLVh7uT8W3Smp2lBwB2pOPn_GztDxRxl59A65m-B4l0dfI9v8rS0

In [11]:
!wget -O negative.csv https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

--2024-04-22 19:02:06--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2024-04-22 19:02:06--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6e03d494f4ca37504359bac8a9.dl.dropboxusercontent.com/cd/0/inline/CRi3wde0tlh3nwCe1AQAwFRotbQsDLLLp2gndVmtyTyUS4PikSLKPNy3z0nSlP4e__WpJbihzzr4ezalEwGOffEDDqup9HUA0KUJ3s1b_U5nzOB_HyQyDs8D0wWlHx7mPvQ/file# [following]
--2024-04-22 19:02:06--  https://uc6e03d494f4ca37504359bac8a9.dl.dropboxusercontent.com/cd/0/inline/CRi3wde0tlh3nwCe1AQAwFRotbQsDLLLp2gndVmtyTyUS4PikSLKPNy3z0nSlP4e__WpJbihzzr4ezalEwGOffEDDqup9HUA0KUJ3s1b_U5nzOB_HyQyDs

In [14]:
import pandas as pd # библиотека для удобной работы с датафреймами
# загрузим и посмотрим на наш датасет

# загружаем положительные твитты
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive) # расставляем метки

# загружаем отрицательные твитты
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative) # расставляем метки

# соединяем два набора данных
df = positive._append(negative)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [15]:
len(df)

226834

Проведем стандартный препроцессинг:

In [16]:
! pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=e012131e10f03dd57b5cd6233f43fa49396d9aa3547126b95f651c6cb9506325
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [17]:
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

# pymorphy2 - библиотека методов для морфологического анализа (в том числе лемматизации) русскоязычного текста
m = pymorphy2.MorphAnalyzer()

# убираем все небуквенные символы
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [18]:
#@lru_cache(maxsize=128)
# если вы работаете не колабе, можно заменить pymorphy на mystem и раскомментирвать первую строку про lru_cache
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "

In [19]:
def clean_text(text):
    return lemmatize(words_only(text))

In [20]:
# распараллелим процесс на 8 копий, чтобы ускорить,
# и к каждому объекту датасета ( = твиту) применим написанную выше функцию препроцессинга

with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))

df['lemmas'] = lemmas
df.head()

<ipython-input-20-c11f18d5da2c>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))


  0%|          | 0/226834 [00:00<?, ?it/s]

,text,label,lemmas
0,"@first_timee хоть я и школота, но поверь, у на...",positive,first_timee хоть я и школотый но поверь у мы т...
1,"Да, все-таки он немного похож на него. Но мой ...",positive,да всё таки он немного похожий на он но мой ма...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,rt katiacheh: ну ты идиотка) я испугаться за т...
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,rt digger : кто то в угол сидеть и погибать от...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,irina_dyshkant вот что значит страшилка :d но ...


Запишем полученные данные в формате для обучения классификатора:

In [21]:
# переводим данные из датафрейма в списки

X = df.lemmas.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

# разбиваем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 151978
total test examples 74856


In [22]:
X_train[4]

'пздца че на улица твориться о о метеееель((('

Далее мы используем для классификации библиотеку fasstext, для этого ей нужно подать данные на вход в особенном формате: текстовый файл, в котором одна строка - один объект выборки, в формате

__label__ 0 первый текст

__label__ 1 второй текст

__label__ 0 третий текст
и т.д.

Записываем train и test выборки в файлы в соответствии с форматом выше:

In [23]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')


with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

Обучаем классификатор fasttext:

In [24]:
! pip install fastText
import fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fastText: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227148 sha256=c15e3bedf8a6cc87f9bc257b0ff8e1062ab167f18bd21db202d77ba0d95bbd03
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fastText


In [25]:

classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')
result

(74856, 0.8971091161697126, 0.8971091161697126)